In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving data.csv to data.csv


In [27]:
import pandas as pd #used for DATAFrames and DataFrames can hold different types data of multidimensional arrays. 
import numpy as np#Numpy provides robust data structures for efficient computation of multi-dimensional arrays & matrices.
import pickle
import matplotlib.pyplot as plt
import sklearn.ensemble as ske 
from sklearn.model_selection import cross_validate
from sklearn import  tree, linear_model
from sklearn.feature_selection import SelectFromModel
from sklearn.externals import joblib
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report

In [18]:

data = pd.read_csv('data.csv', sep='|') #generate df as data
X = data.drop(['Name', 'md5', 'legitimate'], axis=1).values #now droping some coloumns as axis 1(mean coloumn) and will show the values in the rows
y = data['legitimate'].values #values of legitimate data

print('Researching important feature based on %i total features\n' % X.shape[1])# shape() is use in pandas to give number of row/column



Researching important feature based on 54 total features



In [19]:

# Feature selection using Trees Classifier
fsel = ske.ExtraTreesClassifier().fit(X, y)
model = SelectFromModel(fsel, prefit=True)
X_new = model.transform(X)#now features are only 9 :)
nb_features = X_new.shape[1]#will save value 13 as shape is (138047, 13) :}



In [20]:

X_train, X_test, y_train, y_test = train_test_split(X_new, y ,test_size=0.2)#now converting in training and testing data in 20% range hahhahaha ! as total x is 138047 and testing is 138047*0.2=27610 :)
features = []

print('%i features identified as important:' % nb_features) #as mentioned above



13 features identified as important:


In [21]:
indices = np.argsort(fsel.feature_importances_)[::-1][:nb_features]
for f in range(nb_features):
    print("%d. feature %s (%f)" % (f + 1, data.columns[2+indices[f]], fsel.feature_importances_[indices[f]]))

# mean adding to the empty 'features' array the 'important features'
for f in sorted(np.argsort(fsel.feature_importances_)[::-1][:nb_features]):#[::-1] mean start with last towards first 
    features.append(data.columns[2+f])


1. feature DllCharacteristics (0.163785)
2. feature Machine (0.112228)
3. feature Characteristics (0.102347)
4. feature VersionInformationSize (0.074404)
5. feature Subsystem (0.066925)
6. feature SectionsMaxEntropy (0.057215)
7. feature ResourcesMaxEntropy (0.047730)
8. feature ImageBase (0.042364)
9. feature MajorSubsystemVersion (0.039980)
10. feature SizeOfOptionalHeader (0.035455)
11. feature SizeOfStackReserve (0.031465)
12. feature ResourcesMinEntropy (0.030409)
13. feature MajorOperatingSystemVersion (0.024609)


In [22]:
algorithms = {
        "DecisionTree": tree.DecisionTreeClassifier(max_depth=10),
        #The max_depth parameter denotes maximum depth of the tree.
    
        "RandomForest": ske.RandomForestClassifier(n_estimators=50),#In case, of random forest, these ensemble classifiers are            the randomly created decision trees. Each decision tree is a single classifier and the target prediction is based on            the majority voting method.
         #n_estimators ==The number of trees in the forest.
    
        "GradientBoosting": ske.GradientBoostingClassifier(n_estimators=50),
        
        "AdaBoost": ske.AdaBoostClassifier(n_estimators=100),
         #Ada mean Adaptive
         #Both are boosting algorithms which means that they convert a set of weak learners into a single strong learner. They            both initialize a strong learner (usually a decision tree) and iteratively create a weak learner that is added to the            strong learner. They differ on how they create the weak learners during the iterative process.

    }


In [28]:

results = {}
print("\nNow testing algorithms")
for algo in algorithms:
    clf = algorithms[algo]
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    score = clf.score(X_test, y_test)
    print("%s : %f %%" % (algo, score*100))
    results[algo] = score
    print(confusion_matrix(y_test, pred))
    print(classification_report(y_test, pred))

winner = max(results, key=results.get)
print('\nWinner algorithm is %s with a %f %% success' % (winner, results[winner]*100))



Now testing algorithms
DecisionTree : 99.127128 %
[[19118   114]
 [  127  8251]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     19232
           1       0.99      0.98      0.99      8378

    accuracy                           0.99     27610
   macro avg       0.99      0.99      0.99     27610
weighted avg       0.99      0.99      0.99     27610

RandomForest : 99.420500 %
[[19137    95]
 [   65  8313]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19232
           1       0.99      0.99      0.99      8378

    accuracy                           0.99     27610
   macro avg       0.99      0.99      0.99     27610
weighted avg       0.99      0.99      0.99     27610

GradientBoosting : 98.819268 %
[[19081   151]
 [  175  8203]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     19232
           1       0.98      0.98  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
